In [ ]:
import json
import os
import sys
import warnings
from pathlib import Path
from typing import Any, Coroutine, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Prevents ruff from removing the unused module import
_ = [Coroutine, Path, Literal, json]

In [3]:
go_up_from_current_directory(go_up=1)

from src.config import app_settings  # noqa: E402 # type: ignore
from src.utilities.model_config import RemoteModel  # noqa: E402 # type: ignore

settings = app_settings

/Users/mac/Desktop/Projects/smart-rag


In [4]:
from langchain_openai import ChatOpenAI

remote_llm = ChatOpenAI(
    api_key=settings.OPENROUTER_API_KEY.get_secret_value(),  # type: ignore
    base_url=settings.OPENROUTER_URL,
    temperature=0.0,
    seed=1,
    model=RemoteModel.GPT_OSS_120B,
)


# Test the LLMs
response = remote_llm.invoke("Tell me a very short joke.")
response.pretty_print()

================================== Ai Message ==================================

Why don’t scientists trust atoms?  

Because they make up everything.


In [5]:
console.print(response)

AIMessage(
    content='Why don’t scientists trust atoms?  \n\nBecause they make up everything.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 41,
            'prompt_tokens': 74,
            'total_tokens': 115,
            'completion_tokens_details': {
                'accepted_prediction_tokens': None,
                'audio_tokens': None,
                'reasoning_tokens': 19,
                'rejected_prediction_tokens': None,
                'image_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0},
            'cost': 1.5444e-05,
            'is_byok': False,
            'cost_details': {
                'upstream_inference_cost': None,
                'upstream_inference_prompt_cost': 7.4e-06,
                'upstream_inference_completions_cost': 8.2e-06
            }
        },
        'model_provider': 'openai',
        'model_name': 'openai/gpt-oss-120b',
        'system_fingerprint': None,
        'id': 'gen-1767111263-JATNAGkTVmM4Z0cwYihw',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='lc_run--11fea449-66f4-4fe0-8cbb-783f1bbab7ec-0',
    usage_metadata={
        'input_tokens': 74,
        'output_tokens': 41,
        'total_tokens': 115,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'reasoning': 19}
    }
)

In [6]:
import asyncio

import uvloop

# Use Uvloop's implementation (Place this at the entrypoint)
asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())

In [29]:
from typing import Generic

from pydantic import BaseModel, ConfigDict, RootModel

from src.schemas.types import T


class ModelList(BaseModel, Generic[T]):
    """Generic container for lists of Pydantic BaseModel objects.

    This class provides type-safe handling of any BaseModel subclass,
    including LangChain Documents, with validation and utility methods.

    Parameters
    ----------
    items : list[T]
        List of BaseModel instances.

    Examples
    --------
    >>> # With Documents
    >>> docs = ModelList[Document](items=[
    ...     Document(page_content="Hello", metadata={"source": "test"}),
    ...     Document(page_content="World", metadata={"source": "test2"})
    ... ])

    >>> # With custom Pydantic models
    >>> class User(BaseModel):
    ...     name: str
    ...     age: int
    >>>
    >>> users = ModelList[User](items=[
    ...     User(name="Alice", age=30),
    ...     User(name="Bob", age=25)
    ... ])
    """

    model_config = ConfigDict(arbitrary_types_allowed=True)

    items: list[T]

    def __len__(self) -> int:
        """Return the number of items."""
        return len(self.items)

    def __getitem__(self, index: int) -> T:
        """Get item by index."""
        return self.items[index]

    def append(self, item: T) -> None:
        """Add an item to the list."""
        self.items.append(item)

    def extend(self, items: list[T]) -> None:
        """Extend the list with multiple items."""
        self.items.extend(items)

    def to_dicts(self) -> list[dict[str, Any]]:
        """Convert all items to dictionaries."""
        return [
            item.model_dump() if hasattr(item, "model_dump") else dict(item)
            for item in self.items
        ]

    @classmethod
    def from_dicts(
        cls, data: list[dict[str, Any]], model_class: type[T]
    ) -> "ModelList[T]":
        """Create a ModelList from a list of dictionaries.

        Parameters
        ----------
        data : list[dict[str, Any]]
            List of dictionaries to convert.
        model_class : type[T]
            The Pydantic model class to instantiate.

        Returns
        -------
        ModelList[T]
            A new ModelList instance.
        """
        items = [model_class(**item) for item in data]
        return cls(items=items)


class ModelListUpdated(RootModel[list[T]], Generic[T]):
    """Generic container for lists of Pydantic BaseModel objects.

    This class provides type-safe handling of any BaseModel subclass,
    including LangChain Documents, with validation and utility methods.

    Parameters
    ----------
    items : list[T]
        List of BaseModel instances.

    Examples
    --------
    >>> # With Documents
    >>> docs = ModelList[Document](items=[
    ...     Document(page_content="Hello", metadata={"source": "test"}),
    ...     Document(page_content="World", metadata={"source": "test2"})
    ... ])

    >>> # With custom Pydantic models
    >>> class User(BaseModel):
    ...     name: str
    ...     age: int
    >>>
    >>> users = ModelList[User](items=[
    ...     User(name="Alice", age=30),
    ...     User(name="Bob", age=25)
    ... ])
    """

    model_config = ConfigDict(arbitrary_types_allowed=True)

    root: list[T]

    def __len__(self) -> int:
        """Return the number of items."""
        return len(self.root)

    def __getitem__(self, index: int) -> T:
        """Get item by index."""
        return self.root[index]

    def append(self, item: T) -> None:
        """Add an item to the list."""
        self.root.append(item)

    def extend(self, items: list[T]) -> None:
        """Extend the list with multiple items."""
        self.root.extend(items)

    def to_dicts(self) -> list[dict[str, Any]]:
        """Convert all items to dictionaries."""
        return self.model_dump()

    @classmethod
    def from_dicts(
        cls, data: list[dict[str, Any]], model_class: type[T]
    ) -> "ModelList[T]":
        """Create a ModelList from a list of dictionaries.

        Parameters
        ----------
        data : list[dict[str, Any]]
            List of dictionaries to convert.
        model_class : type[T]
            The Pydantic model class to instantiate.

        Returns
        -------
        ModelList[T]
            A new ModelList instance.
        """
        items = [model_class(**item) for item in data]
        return cls(root=items)


class User(BaseModel):
    name: str
    age: int

In [30]:
user_1 = User(name="Alice", age=30)
user_2 = User(name="Bob", age=25)
my_obj = ModelList[User](items=[user_1, user_2])
my_obj

ModelList[User](items=[User(name='Alice', age=30), User(name='Bob', age=25)])

In [31]:
my_obj.append(User(name="Charlie", age=28))

my_obj

ModelList[User](items=[User(name='Alice', age=30), User(name='Bob', age=25), User(name='Charlie', age=28)])

In [33]:
other_obj = ModelListUpdated[User](root=[user_1])
other_obj.model_dump(), other_obj.to_dicts()

([{'name': 'Alice', 'age': 30}], [{'name': 'Alice', 'age': 30}])

In [34]:
my_obj.model_dump()

{'items': [{'name': 'Alice', 'age': 30},
  {'name': 'Bob', 'age': 25},
  {'name': 'Charlie', 'age': 28}]}

In [42]:
a = [{"name": "Bob", "age": 25}]
b = [{"name": "Alice", "age": 30}]
list(dict.fromkeys(a + b))

TypeError: unhashable type: 'dict'

In [ ]:
a + b

[{'name': 'Bob', 'age': 25}, {'name': 'Alice', 'age': 30}]